In [56]:
import pandas as pd
import datetime

In [57]:
df = pd.read_excel('data.xlsx')

In [58]:
df = df.drop('month', axis=1)

In [59]:
df.head(10)

,merchant_name,day,cashback
0,1С Интерес,2022-01-21 00:00:00,240.53
1,1С Интерес,2022-01-22 00:00:00,399.21
2,1С Интерес,2022-01-23 00:00:00,385.44
3,1С Интерес,2022-01-24 00:00:00,209.48
4,1С Интерес,2022-01-25 00:00:00,361.80
5,1С Интерес,2022-01-26 00:00:00,380.77
6,1С Интерес,2022-01-27 00:00:00,2163.35
7,1С Интерес,2022-01-28 00:00:00,1053.93
8,1С Интерес,2022-01-29 00:00:00,534.87
9,1С Интерес,2022-01-30 00:00:00,458.59


In [60]:
# Получает на вход год, месяц, день и возвращает день недели в виде его номера
def get_weekday(year, month, day):
    return datetime.date(year, month, day).weekday()

# Получает на вход строку вида "YYYY-MM-DD hh:mm:ss" и возвращает year, month, day
def get_date(date_string):
    date = str(date_string).split(' ')[0]
    year, month, day = map(int, date.split('-'))
    return year, month, day

# Переведи стоец day формата "YYYY-MM-DD hh:mm:ss" в столбец с результатом get_date, но в виде строки через пробел
df['day'] = df['day'].apply(lambda x : ' '.join(list(map(str, get_date(x)))))
df['weekday'] = df['day'].apply(lambda x : get_weekday(*map(int, x.split(' '))))
df['is_weekend'] = df['weekday'].apply(lambda x : 1 if x in [5, 6] else 0)
df

,merchant_name,day,cashback,weekday,is_weekend
0,1С Интерес,2022 1 21,240.53,4,0
1,1С Интерес,2022 1 22,399.21,5,1
2,1С Интерес,2022 1 23,385.44,6,1
3,1С Интерес,2022 1 24,209.48,0,0
4,1С Интерес,2022 1 25,361.80,1,0
...,...,...,...,...,...
27537,Яндекс.Еда и Лавка,2023 1 27,13950.84,4,0
27538,Яндекс.Еда и Лавка,2023 1 28,17400.74,5,1
27539,Яндекс.Еда и Лавка,2023 1 29,13286.25,6,1
27540,Яндекс.Еда и Лавка,2023 1 30,8075.79,0,0
